Übung: ConfigMap
---------------------------

Am Beispiel von Apache Web Server soll der Einsatz von ConfigMap Demonstriert werden.

Zuerst müssen die ConfigMaps mittels `kubectl` erstellt werden. Vorher erstellen wir eine neue Namespace `configmap` um Erstellen Ressourcen von den anderen zu trennen:

In [ ]:
! kubectl create namespace configmap
! kubectl create configmap web2 --from-file=index=09-8-ConfigMap/index-2.html -n configmap
! kubectl create configmap web1 --from-file=index=09-8-ConfigMap/index-1.html -n configmap


Die so erstellen ConfigMaps, hätten wir auch im YAML Format beschreiben können. Deshalb geben wir uns diese im YAML Format aus:

In [ ]:
! kubectl get configmap -o yaml

Anschliessend sind die Volumes in der YAML Datei so zu setzen, dass sie auf die ConfigMap Keys zugreifen

- - -

    apiVersion: v1
    kind: Pod
    metadata:
      labels:
        app.kubernetes.io/name: web1
      name: web1
      namespace: configmap  
    spec:
      containers:
      - image: httpd
        name: apache
        volumeMounts:    
         - name: config-volume
           mountPath: /usr/local/apache2/htdocs
      volumes:
        - name: config-volume
          configMap:
            name: web1
            items:
            - key: index
              path: index.html

- - -

Dann können wir die Pods und Services erstellen:

In [ ]:
! kubectl apply -f 09-8-ConfigMap

Die Ausgaben der Web Server mit den jeweiligen `index.html` Dateien bzw. ConfigMap schauen wir uns an:

In [ ]:
! echo "Web Server 1: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service web1 -n configmap -o=jsonpath='{ .spec.ports[0].nodePort }')/")
! echo "Web Server 2: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service web2 -n configmap -o=jsonpath='{ .spec.ports[0].nodePort }')/")


Und wir schauen uns Verzeichnis und Dateiinhalt an, wie die Pods diese sehen:

In [ ]:
! kubectl exec -n configmap web1 -- ls -la /usr/local/apache2/htdocs

In [ ]:
! kubectl exec -n configmap web1 -- cat /usr/local/apache2/htdocs/index.html

Aufräumen

In [ ]:
! kubectl delete namespace configmap